<a href="https://colab.research.google.com/github/gagandt/modelling-air-pollution/blob/master/multivariate/GRID_CNN_on_PM2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from pandas import DataFrame
import pandas
from pandas import concat

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [3]:
import os
from pandas import read_csv
from datetime import datetime
# load data
os.chdir('/content/drive/My Drive/Colab Notebooks')
def parse(x):
	return datetime.strptime(x, '%Y %m %d %H')
dataset = read_csv('./data/beijing.csv',  parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=parse)
dataset.drop('No', axis=1, inplace=True)
# manually specify column names
dataset.columns = ['pollution', 'dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain']
dataset.index.name = 'date'
# mark all NA values with 0
dataset['pollution'].fillna(0, inplace=True)
# drop the first 24 hours
dataset = dataset[24:]
# summarize first 5 rows
print(dataset.head(5))
# save to file
dataset.to_csv('pollution.csv')


                     pollution  dew  temp   press wnd_dir  wnd_spd  snow  rain
date                                                                          
2010-01-02 00:00:00      129.0  -16  -4.0  1020.0      SE     1.79     0     0
2010-01-02 01:00:00      148.0  -15  -4.0  1020.0      SE     2.68     0     0
2010-01-02 02:00:00      159.0  -11  -5.0  1021.0      SE     3.57     0     0
2010-01-02 03:00:00      181.0   -7  -5.0  1022.0      SE     5.36     1     0
2010-01-02 04:00:00      138.0   -7  -5.0  1022.0      SE     6.25     2     0


In [32]:
import copy
import os
from math import sqrt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
import numpy as np

def grid_search(dataset, lookback, filters, kernels, batch_sizes, epochs, split_percentage):
#   os.chdir('MLP')
  scores = list()
  
  for lookback_period in lookback:
    values = series_to_supervised(dataset, lookback_period, 1).values
    
    n_train_hours = (int)(split_percentage * len(values))
    train = values[:n_train_hours, :]
    test = values[n_train_hours:, :]
    
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    
    n_features = 8
    n_hours = lookback_period
    n_obs = n_hours * n_features
    train_X, train_y = train[:, :n_obs], train[:, -n_features]
    test_X, test_y = test[:, :n_obs], test[:, -n_features]

    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
    test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    
    for number_of_filters in filters:      
        for kernel_size in kernels:
          model = Sequential()
          try:
            model.add(Conv1D(filters=number_of_filters, kernel_size=kernel_size, activation='relu', input_shape=(n_hours, n_features)))
            model.add(MaxPooling1D(pool_size=kernel_size))
            model.add(Flatten())

            model.add(Dense(1))
            model.compile(loss='mae', optimizer='adam')

            ref_model = copy.deepcopy(model)

            for batch_size in batch_sizes:
              for number_of_epochs in epochs:         
                model = copy.deepcopy(ref_model)

                name = str(lookback_period)+'_'+str(number_of_filters)+'_'+str(kernel_size)+'_'+str(batch_size)+'_'+str(number_of_epochs)
                print("Starting training for : ", name)

                history = model.fit(train_X, train_y, epochs=number_of_epochs, batch_size=batch_size, validation_data=(test_X, test_y), verbose=2, shuffle=False)
                yhat = model.predict(test_X)

                np.save('CNN_runtime/history' + name, history)
                np.save('CNN_runtime/test_y' + name, test_y)
                np.save('CNN_runtime/yhat' + name, yhat)

                # print(yhat)
                # print(test_X)
                # print(test_y)
                rmse = mean_squared_error(yhat, test_y)
                print("RMSE for ", name, " = " , rmse)

                scores.append([rmse, lookback_period, number_of_filters, kernel_size, batch_size, number_of_epochs])
          except:
            print('error')

  
#   os.chdir('..')
  return scores
          
  

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# load dataset
dataset = read_csv('pollution.csv', header=0, index_col=0)
values = dataset.values

# integer encode direction
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])

# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [33]:
lookback = [3,5,7]
filters = [2,4,16, 32, 64]
kernels = [2,3, 5]
# batch_size = [50]
batch_size = [50, 75, 100]
# epochs = [1]
epochs = [50,100]
split_percentage = 0.8

scores = grid_search(values, lookback, filters, kernels, batch_size, epochs, split_percentage)
print(scores)
np.save('CNN_scores', np.array(scores))

(35037, 3, 8) (35037,) (8760, 3, 8) (8760,)
Starting training for :  3_64_2_50_1
Train on 35037 samples, validate on 8760 samples
Epoch 1/1
 - 1s - loss: 24.4942 - val_loss: 20.3787
RMSE for  3_64_2_50_1  =  1068.9691
error
Starting training for :  3_4_2_50_1
Train on 35037 samples, validate on 8760 samples
Epoch 1/1
 - 1s - loss: 31.9270 - val_loss: 21.5732
RMSE for  3_4_2_50_1  =  1343.2654
error
Starting training for :  3_16_2_50_1
Train on 35037 samples, validate on 8760 samples
Epoch 1/1
 - 1s - loss: 44.8895 - val_loss: 18.3046
RMSE for  3_16_2_50_1  =  1034.0762
error
(35036, 5, 8) (35036,) (8759, 5, 8) (8759,)
Starting training for :  5_64_2_50_1
Train on 35036 samples, validate on 8759 samples
Epoch 1/1
 - 1s - loss: 28.1460 - val_loss: 20.1438
RMSE for  5_64_2_50_1  =  1124.1605
Starting training for :  5_64_3_50_1
error
Starting training for :  5_4_2_50_1
Train on 35036 samples, validate on 8759 samples
Epoch 1/1
 - 1s - loss: 191.7950 - val_loss: 95.9669
RMSE for  5_4_2_50_